<a href="https://colab.research.google.com/github/rihai/VideoSwitchingScore/blob/main/VideoSwitchingScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#! pip install numpy==1.23.1
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

import tensorflow as tf
#import tensorflow-gpu
import random

import time # for fps calculation
import os
import threading

import torch

from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms


from multiprocessing.pool import ThreadPool
from collections import deque

#from common import clock, draw_str, StatValue

# install mediapipe
!pip install mediapipe

# clone github code
!git clone https://github.com/cedro3/mediapipe.git
%cd mediapipe/
# initial setting
import mediapipe as mp



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 50.1 MB/s eta 0:00:00
Cloning into 'mediapipe'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 162 (delta 62), reused 49 (delta 49), pack-reused 84
Receiving objects: 100% (162/162), 2.40 MiB | 29.58 MiB/s, done.
Resolving deltas: 100% (86/86), done.
/content/mediapipe


In [ ]:
v1 = cv2.VideoCapture('/content/drive/MyDrive/MasterThesis/SAT_cam1_short_audio.mp4')
v2 = cv2.VideoCapture('/content/drive/MyDrive/MasterThesis/SAT_cam2_short_audio.mp4')
v3 = cv2.VideoCapture('/content/drive/MyDrive/MasterThesis/SAT_totale_short_audio.mp4')

video1 = '/content/drive/MyDrive/MasterThesis/Stachel01/Stachel01_cam1.mp4'
video2 = '/content/drive/MyDrive/MasterThesis/Stachel01/Stachel01_cam2.mp4'
video3 = '/content/drive/MyDrive/MasterThesis/Stachel01/Stachel01_totale.mp4'

st1 = cv2.VideoCapture('/content/drive/MyDrive/MasterThesis/Footage/Stachel/Stachel01_cam1.mp4')
st2 = cv2.VideoCapture('/content/drive/MyDrive/MasterThesis/Footage/Stachel/Stachel01_cam2.mp4')
st3 = cv2.VideoCapture('/content/drive/MyDrive/MasterThesis/Footage/Stachel/Stachel01_totale.mp4')

fps1 = v1.get(cv2.CAP_PROP_FPS)
fps2 = v2.get(cv2.CAP_PROP_FPS)
fps3 = v3.get(cv2.CAP_PROP_FPS)

frame_count = int(st1.get(cv2.CAP_PROP_FRAME_COUNT))

w1 = int(st1.get(cv2.CAP_PROP_FRAME_WIDTH)) # width of v1
h1 = int(st1.get(cv2.CAP_PROP_FRAME_HEIGHT)) # height of v2

fourcc = cv2.VideoWriter_fourcc(*'MJPG') # for avi
fourcc_mp4 = cv2.VideoWriter_fourcc(*'MP4V') # for mp4

#output = cv2.VideoWriter('test.mp4',fourcc_mp4, 50.0,(w1,h1))

#check fps of video source, in case of video file
print("fps1: ", fps1, "fps2: ", fps2, "fps3:", fps3)

fps1:  50.0 fps2:  50.0 fps3: 50.0


In [ ]:

mpDraw = mp.solutions.drawing_utils
mpFaceMesh = mp.solutions.face_mesh
faceMesh = mpFaceMesh.FaceMesh(
    static_image_mode = False,
    max_num_faces = 3)
drawSpec = mpDraw.DrawingSpec(thickness=1, circle_radius=1)

In [ ]:
def detect_talking(frame,results):  
  #results = faceMesh.process(img_base)  
  landmarks = results.face_landmarks
 
    
  source = landmarks.landmark[13]
  target = landmarks.landmark[14]
 
  relative_source = (int(frame.shape[1] * source.x), int(frame.shape[0] * source.y))
  relative_target = (int(frame.shape[1] * target.x), int(frame.shape[0] * target.y))
      #print('source: ', relative_source)
      #print('target: ', relative_target)
  delta = relative_source[1] - relative_target[1] # distance between 2 keypoints
  #if(abs(delta) > 0):
   #   isTalking = True
    #  print('Talking in function: ',isTalking)
      #print('delta: ', delta)
      #print('coord1; ', relative_source[1])
      
      #print('coord2; ', relative_target[1])
      #cv2.line(img_base, relative_source, relative_target, (255, 255, 255), thickness = 1)
  #cv2.circle(frame,relative_source, 5, (0,255, 0), 1)
  #cv2.circle(frame,relative_target, 5, (0, 255, 0), 1)
  return abs(delta)
      #return abs(delta)

In [ ]:
def count_time(duration):
  start_time = time.time() # time for counting seconds later
  current_time = time.time()
  if current_time - start_time > duration:
    start_time = current_time
  



In [ ]:


seq = []
#takes current shot as input
def shot_sequence(input): 
  seq.append(input) # adds current shot at the end of the list of previous shots
  print(seq)
  print("length",len(seq))
  if(len(seq)>2): # only keeps the last three shots in the list
    seq.pop(0) # removes oldest shot in list
    
  next_shot = random.randrange(0, 2) # if there is no other criterion outweighing the other one of the previous two shots will be chosen on random
  return seq[next_shot]
  #print(random.randrange(0, 2)) 
  


In [ ]:
def detect_allTalking(results1, results2, results3):
     #start_time = current_time
      #if(current_time - start_time > 10):
                # Check if someone is talking
      #if current_time - start_time > 20:
         
          #print("time delta detection", current_time - start_time)
          if results1.pose_landmarks and results1.face_landmarks:
              
                #mouth_open = abs(int(results1.face_landmarks.landmark[13])-int(results1.face_landmarks.landmark[14]))
               # mouth_open =  detect_talking(frame1, results1)
                if detect_talking(frame1, results1) > 0.04:
                  talking1 = True

          if results2.pose_landmarks and results2.face_landmarks:
                #mouth_open =  abs(int(results2.face_landmarks.landmark[13])-int(results2.face_landmarks.landmark[14]))
                #detect_talking(frame1, results1)
                if detect_talking(frame2, results2) > 0.04:
                  talking2 = True

          if results3.pose_landmarks and results3.face_landmarks:
                #mouth_open =  abs(int(results3.face_landmarks.landmark[13])-int(results3.face_landmarks.landmark[14]))
                if detect_talking(frame3, results3) > 0.04:
                  talking3 = True
        
          
        #  start_time = current_time


I-FRAME EXTRACTION

In [ ]:
import os
import cv2
import subprocess
# /content/mediapipe/SAT_test_cam2_i_frame_0.jpg
filename = '/content/drive/MyDrive/MasterThesis/test_files/SAT_test_cam1.mp4'
filename2 = '/content/drive/MyDrive/MasterThesis/test_files/stachel_2023_02_01_cam1.mp4' #'/content/drive/MyDrive/MasterThesis/test_files/SAT_test_cam2.mp4'
filename3 = '/content/drive/MyDrive/MasterThesis/test_files/SAT_test_totale.mp4'
frames = []
frames2 = []
frames3 = []

def get_frame_types(video_fn):
    command = 'ffprobe -v error -show_entries frame=pict_type -of default=noprint_wrappers=1'.split()
    out = subprocess.check_output(command + [video_fn]).decode()
    frame_types = out.replace('pict_type=','').split()
    return zip(range(len(frame_types)), frame_types)

def save_i_keyframes(video_fn):
    frame_types = get_frame_types(video_fn)
    i_frames = [x[0] for x in frame_types if x[1]=='I']
    if i_frames:
        basename = os.path.splitext(os.path.basename(video_fn))[0] 
        cap = cv2.VideoCapture(video_fn) #vide capture
        for frame_no in i_frames: # 
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no)
            ret, frame = cap.read()
            outname = basename+'_i_frame_'+str(frame_no)+'.jpg'
            cv2.imwrite(outname, frame)
            frames.append(frame) # adds frames to list
            #print("frames in list", frames[0]) 
            print ('Saved: '+outname)
        cap.release()
    else:
        print ('No I-frames in '+video_fn)

if __name__ == '__main__':
    save_i_keyframes(filename)
    #save_i_keyframes(filename2)
    #save_i_keyframes(filename3)

Saved: SAT_test_cam1_i_frame_0.jpg
Saved: SAT_test_cam1_i_frame_50.jpg
Saved: SAT_test_cam1_i_frame_100.jpg
Saved: SAT_test_cam1_i_frame_150.jpg
Saved: SAT_test_cam1_i_frame_200.jpg
Saved: SAT_test_cam1_i_frame_250.jpg
Saved: SAT_test_cam1_i_frame_300.jpg
Saved: SAT_test_cam1_i_frame_350.jpg
Saved: SAT_test_cam1_i_frame_400.jpg
Saved: SAT_test_cam1_i_frame_450.jpg
Saved: SAT_test_cam1_i_frame_500.jpg
Saved: SAT_test_cam1_i_frame_550.jpg
Saved: SAT_test_cam1_i_frame_600.jpg
Saved: SAT_test_cam1_i_frame_650.jpg
Saved: SAT_test_cam1_i_frame_700.jpg
Saved: SAT_test_cam1_i_frame_750.jpg
Saved: SAT_test_cam1_i_frame_800.jpg
Saved: SAT_test_cam1_i_frame_850.jpg
Saved: SAT_test_cam1_i_frame_900.jpg
Saved: SAT_test_cam1_i_frame_950.jpg


In [ ]:
def calculate_score(talking, nodding, gesturing, time): # booleans for talking, nodding, gesturing and time intervals for every video source
  score = 0
  if talking: 
    score = 7

  if nodding & talking != True & int(time) > 7: # TODO calculate switch back time
    score = 7

  if gesturing & talking:
    score = 8

  return "score: " + str(score)






SCORE SYSTEM COMPLETE ATTEMPT

In [ ]:
def detect_nodding(frame, results, old_nod):
  landmarks = results.pose_landmarks
  #old_nod = -1
  #neck = results1.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE]
  #head_top = results1.pose_landmarks.landmark[mp_holistic.PoseLandmark.TOP_HEAD]
                #if head_top.y < neck.y:
 
  source = landmarks.landmark[1] # 4
  #print("landmark 1", source)
  target = landmarks.landmark[14]
  
  relative_source = (int(frame.shape[1] * source.x), int(frame.shape[0] * source.y))
  #if old_nod != -1: # if the old keypoint is not -1 then calculate the delta between current keypoint and old one
  delta = relative_source[0] - old_nod
  #print("old nod in function", old_nod)
  old_nod = relative_source[0] # y coordinate is now stored in temp variable
  #else :
  #  return source #print(old_nod)#"first keypoint"
      
      #print('coord2; ', relative_target[1])
      #cv2.line(img_base, relative_source, relative_target, (255, 255, 255), thickness = 1)
  cv2.circle(frame,relative_source, 5, (0,255, 0), 1)
  return abs(delta)
      #return abs(delta)

In [ ]:
def put_text(frame,text, coord):
  font = cv2.FONT_HERSHEY_SIMPLEX
  coord = (100, 50)
  
  fontScale = 1
   
  color = (255, 255, 255)
  
# Line thickness of 2 px
  thickness = 2
   
# Using cv2.putText() method
  image = cv2.putText(frame, text, coord, font, 
                    fontScale, color, thickness, cv2.LINE_AA)
  return image

Check for consecutive order

In [ ]:
def is_consecutive(frame_numbers):
  if len(frame_numbers) > 1:
    for i in range(len(frame_numbers) - 1):
        if frame_numbers[i+1] != frame_numbers[i] + 1:
            return False
            
    return True


In [ ]:
import cv2
import mediapipe as mp
from google.colab.patches import cv2_imshow
import time




# Set up video capture and output video
#cap1 = cv2.VideoCapture('/content/drive/MyDrive/MasterThesis/Stachel01/Stachel01_cam1.mp4')
#cap2 = cv2.VideoCapture('/content/drive/MyDrive/MasterThesis/Stachel01/Stachel01_cam2.mp4')
#cap3 = cv2.VideoCapture('/content/drive/MyDrive/MasterThesis/Stachel01/Stachel01_totale.mp4')

cap1 = cv2.VideoCapture('/content/drive/MyDrive/MasterThesis/test_files/SAT_test_cam1.mp4')
cap2 = cv2.VideoCapture('/content/drive/MyDrive/MasterThesis/test_files/SAT_test_cam2.mp4')
cap3 = cv2.VideoCapture('/content/drive/MyDrive/MasterThesis/test_files/SAT_test_totale.mp4')

nodding_keypoint = -1

current_time = time.time()
switch_time = 0
detections= 0 #counter for detections within one source #[] # list with detecions in frames, contains only true or false values
nod_det = 0 # counter for detecting nodding in individual frame

i=0 # for iterating over framelist


frames_talking = [] # list with frame indices for detections of talking
frames_nodding = [] # list with frame indices for detections of talking
frames_gesturing = [] # list with frame indices for detections of talking

score = 0
interval = 5 # time interval before switching
old_nod = -1
# Set up Mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic()
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  # Loop through video frames
  start_time = time.time()
  while i < len(frames)-1:
    
      # Process frames with Mediapipe
      results1 = holistic.process(frames[i])#frame1)
     # results2 = holistic.process(frame2)
     # results3 = holistic.process(frame3)
    
     
      # Check if anyone is talking in each frame
      talking1 = False
      talking2 = False
      talking3 = False

      nodding1 = False
      old_i = i


      gesturing1 = False

      old_frame = 0
      #current_frame = cap1.get(cv2.CAP_PROP_POS_FRAMES); # current frame of the video source
      #print("current_frame", current_frame)
   
      #checks if someone is talking
      if results1.pose_landmarks and results1.face_landmarks:
               
        if detect_talking(frames[i], results1) > 0.04: # if a certain threshold is reached --> talking
          frames_talking.append(i) # list with frame indices
          is_consecutive(frames_talking) # checks if frames added are consecutive
          print("consecutive",is_consecutive(frames_talking))

          detections = detections + 1 # if an open mouth is detected in a keyframe increase the counter 
          #old_i = i
          if detections > 5 and is_consecutive(frames_talking): # if a certain threshold is passed set talking to True and reset the counter
           if i < old_i+5: 
            talking1 = True
            print("talking + i ", talking1, i)

           detections = 0
           old_i = i
           frames_talking.clear() # emtpies list if the numbers are not consecutive
         
          
      # checks if someone is nodding
      if results1.pose_landmarks:
           landmarks = results1.pose_landmarks
  
           source = landmarks.landmark[0] # 4
           relative_source = (int(frames[i].shape[1] * source.x), int(frames[i].shape[0] * source.y)) # calculates coordinates in pixel values
          
           delta = relative_source[0] - old_nod
           old_nod = relative_source[0] # y coordinate is now stored in temp variable

           if delta > 15: # if delta is over a certain threshold --> nodding

            nod_det = nod_det + 1
            #print("nod counter", nod_det)
            
           #check for consecutive frames
           if nod_det > 3 & talking1 != True: # if the delta occurs in consecutive frames
             nodding1 = True
             print("nodding", nodding1)

           #cv2.circle(frames[i],relative_source, 5, (0,255, 0), 1)
     
          

       

      # Check if someone is gesturing
      if results1.left_hand_landmarks or results1.right_hand_landmarks:
                #gesturing_count += 1
                print("visible hands", i)
                gesturing = True

      # use frame number for calculting time interval!!!!!!!!!!!!!!!!!!!!!!!!!
      
      cv2_imshow(put_text(frames[i], "i: "+str(i), (100, 50))) #str(calculate_score(talking1, nodding1, gesturing1, time.time()))))#"nodding "+str(nodding1)+" talking "+ str(talking1))) #writes text on image
      # save written frames to drive
      out_path = "/content/drive/MyDrive/results/frame_tests"
      frame_name = 'Frame'+str(i)+'.jpg'
      cv2.imwrite(os.path.join(out_path, frame_name), frames[i])
      i=i+1      

      #elif results2.pose_landmarks and results2.face_landmarks:
               
         # if detect_talking(frame2, results2) > 0.04:
                    
       
                    
                  

      
     # elif results3.pose_landmarks and results3.face_landmarks:
                            #mouth_open =  abs(int(results3.face_landmarks.landmark[13])-int(results3.face_landmarks.landmark[14]))
                   
                  #if detect_talking(frame3, results3) > 0.04:
                               
                  
                              
        #j=j+1         

      #if results1.pose_landmarks and results1.face_landmarks and results1.left_hand_landmarks and results1.right_hand_landmarks:
       #   talking1 = any([landmark.visibility > 0.5 for landmark in results1.face_landmarks.landmark])
      #if results2.pose_landmarks and results2.face_landmarks and results2.left_hand_landmarks and results2.right_hand_landmarks:
      #    talking2 = any([landmark.visibility > 0.5 for landmark in results2.face_landmarks.landmark])
     # if results3.pose_landmarks and results3.face_landmarks and results3.left_hand_landmarks and results3.right_hand_landmarks:
    #    talking3 = any([landmark.visibility > 0.5 for landmark in results3.face_landmarks.landmark])


